In [1]:
import json
import re

import requests
from flask import Flask
from flask import request

from paper.config import ANALYZER_PATH, ANALYZER_PORT
from paper.config import LANGDETECT_PORT, LANGDETECT_PATH, STEMMER_PORT, STEMMER_PATH
from paper.analyze import normalize
from paper.utils import post_request


app = Flask(__name__)

regex = re.compile('[!,/\().%]')

@app.route(ANALYZER_PATH, methods=['GET', 'POST'])
def analyze():
    """
    Analyzer provide stemm, tokenize and lang detect workflow
    
    Receive: list of lists of textes
    Return: list of lists of stemmed tokens for each document + languages
    """
    if request.method=='POST':
        data = request.json
        
        textes = data["textes"]
        textes = [normalize(text) for text in textes]
        
        textes_json = {'textes' : textes}
        r_lang = post_request(LANGDETECT_PORT, LANGDETECT_PATH, textes_json)
        langs = r_lang['langs']
        
        stemm_json = {'textes' : textes, 'langs': langs}
        r_stemmed = post_request(STEMMER_PORT, STEMMER_PATH, stemm_json)
        stemmed_docs = r_stemmed['stemmed']

        return json.dumps({'status':'ok',
                           'langs': langs,
                           'stemmed_docs': stemmed_docs})
    else:
        return "post json {textes: 'your text here'}"
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=ANALYZER_PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13571/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Nov/2018 13:00:32] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:02] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:32] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:02:49] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:25:31] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:26:06] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:26:10] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:28:03] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:29:22] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:32:25] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 13:32:35] "POST /analyze HTTP/1.1" 200 -
